# GPT-5.1 와 API 사용 예

| 모드 (Mode)      | 용도 (Use Case)   | 속도 (Speed, 1~5) | 정확도 (Accuracy, 1~5) | 비용 (Cost, 1~5) |
|------------------|--------------------|---------------------|--------------------------|-------------------|
| **Instant**       | 일상 작업          | 3                   | 4                        | 2                 |
| **Thinking**      | 복잡한 추론        | 2                   | 5                        | 3                 |
| **No Reasoning**  | 빠른 응답          | 4                   | 3                        | 1                 |


✅ GPT-5.1 API 모델명 (Instant / Thinking)

**Instant 모드**
- 모델명: "gpt-5.1-chat-latest"
- 특징: 빠른 응답, 일상적인 작업·대화형 작업에 적합
- reasoning_effort="high" 옵션을 주면 Instant 모델에서도 일정 수준의 Thinking 스타일 추론을 유도할 수 있음

**Thinking 모드**
- 모델명: "gpt-5.1"
- 특징: 복잡한 추론, 논리적 문제 해결, 코드 분석에 적합

https://platform.openai.com/docs/pricing

https://platform.openai.com/docs/models


In [1]:
# 1. OpenAI Python 라이브러리 설치
!pip install openai python-dotenv

In [2]:
import os
from getpass import getpass

# OpenAI API key 입력
openai_api_key = getpass("Enter yout OpenAI API key : ")


# API키 설정
os.environ["OPENAI_API_KEY"] = openai_api_key

### **API Key 발급**

1. https://platform.openai.com 접속
2. API Keys 메뉴에서 새 키 생성

In [5]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 간단한 테스트
response = client.chat.completions.create(
    model="gpt-5.1-chat-latest",
    messages=[{"role": "user", "content": "안녕!"}]
)
print(response.choices[0].message.content)

안녕하세요! 무엇을 도와드릴까요? 😊


**OpenAI Key Pricing**
https://platform.openai.com/docs/pricing


##  Instant 모드

### 1. Adaptive Reasoning - 질문 난이도 자동 판단
- GPT-5.1 Instant는 질문의 난이도를 자동으로 판단
- 쉬운 질문: 빠른 응답 (2초)
- 어려운 질문: 깊은 사고 (10초)



In [6]:
# 쉬운 질문 - 빠른 응답
easy_question = "npm 전역 패키지 목록 명령어는?"
# 예상 응답시간 : 2초

# API call
response = client.chat.completions.create(
    model="gpt-5.1-chat-latest",
    messages=[{"role": "user", "content": easy_question}]
)

# 응답 확인
print(response.choices[0].message.content)

npm 전역 패키지 목록을 확인하는 명령어는 다음입니다.

```
npm list -g --depth=0
```

설명:
- `-g` : 전역(global) 패키지를 의미
- `--depth=0` : 최상위 패키지만 표시 (의존성 트리 전체를 출력하지 않음)

추가로 설치된 전역 패키지 경로를 확인하려면:

```
npm root -g
```

전역 npm의 설치 위치 확인:

```
npm prefix -g
```


In [8]:
# 어려운 질문 - 깊은 사고
hard_question = """
다음 시나리오에서 최적의 마이크로서비스 아키텍처를 설계하세요:
- 일일 1억 건의 트랜젝션
- 99.99% 가용성 요구
- 실시간 데이터 동기화
- 글로벌 배포
"""

# API call
response = client.chat.completions.create(
    model='gpt-5.1-chat-latest',
    messages=[{"role": "user", "content": hard_question}]
)

print(response.choices[0].message.content)

아래는 주어진 요구사항(일일 1억 트랜잭션, 99.99% 가용성, 실시간 동기화, 글로벌 배포)을 만족하는 **최적 수준의 마이크로서비스 아키텍처 설계안**입니다.

-------------------------------------
최상위 목표
-------------------------------------
• 대규모 트래픽을 안정적으로 처리  
• 진짜 실시간 글로벌 데이터 일관성  
• 장애 격리 및 자동 복구  
• 지연 최소화와 지역별 독립적 확장  

-------------------------------------
아키텍처 구성
-------------------------------------

1. 서비스 분리 구조  
• API Gateway  
  - 글로벌 Anycast 기반 라우팅  
  - 인증, 레이트 리밋, Geo routing  
• Domain 별 마이크로서비스  
  - 완전한 독립 배포 및 스케일링  
  - 각 서비스는 Stateless 우선 설계  
• Event Processing 서비스  
  - 핵심 비즈니스 이벤트 처리  
  - CQRS, Event Sourcing 적용 시 확장 유리  
• Background Worker  
  - 지연 허용 작업 처리  

2. 데이터 아키텍처  
• 글로벌 분산 DB  
  - Google Spanner, AWS Aurora Global Database, CockroachDB 등  
  - 강한 일관성 + 낮은 지연 글로벌 리드 가능  
• Local Write, Global Sync 패턴  
  - 지역별 Primary 노드  
  - 글로벌 consensus replica  
• 서비스별 독립 DB 원칙  
  - 마이크로서비스 간 데이터 공유 금지  
  - 필요 시 이벤트 기반 동기화  

3. 메시징 및 실시간 데이터 동기화  
• 글로벌 메시지 브로커  
  - Kafka Global Cluster 또는 Pulsar Geo-replication  
  - T

### 2. 실전 활용

**Use Case 1: 이메일 자동 작성**


In [9]:
prompt = """
다음 정보로 전문적인 이메일을 작성하세요:
- 받는 사람: 김철수 부장님
- 목적: 프로젝트 일정 연기 요청
- 이유: 외부 API 통합 지연
- 새로운 일정: 2주연장
"""

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{'role':'user','content':prompt}]
)

print(response.choices[0].message.content)

제목: 프로젝트 일정 연기 요청드립니다(외부 API 연동 지연)

김철수 부장님께,

안녕하세요. 현재 진행 중인 [프로젝트명]의 일정과 관련하여 부장님께 보고 드리며, 일정 연기를 요청드립니다.

사유: 외부 API 통합 작업이 지연되어 기존 일정에 차질이 발생합니다. 이로 인해 핵심 기능의 안정적 통합 및 검증에 추가 시간이 필요합니다.

제안 일정: 새로운 일정은 기존 마감일에서 2주 연장하는 방향으로 조정하고자 합니다.

다음 단계: 연장된 일정에 맞춰 리스크 관리 및 자원 재배치를 포함한 변경 계획을 마련하겠습니다. 필요하시면 회의를 통해 자세한 조정안을 함께 논의하고자 합니다. 편하신 시간에 일정 조정 가능하도록 회의 일정도 맞추겠습니다.

감사합니다.

발신자 드림


In [10]:
response

ChatCompletion(id='chatcmpl-CgKCKq8FUVzZ5jiM9O9RL5ASbYHuK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='제목: 프로젝트 일정 연기 요청드립니다(외부 API 연동 지연)\n\n김철수 부장님께,\n\n안녕하세요. 현재 진행 중인 [프로젝트명]의 일정과 관련하여 부장님께 보고 드리며, 일정 연기를 요청드립니다.\n\n사유: 외부 API 통합 작업이 지연되어 기존 일정에 차질이 발생합니다. 이로 인해 핵심 기능의 안정적 통합 및 검증에 추가 시간이 필요합니다.\n\n제안 일정: 새로운 일정은 기존 마감일에서 2주 연장하는 방향으로 조정하고자 합니다.\n\n다음 단계: 연장된 일정에 맞춰 리스크 관리 및 자원 재배치를 포함한 변경 계획을 마련하겠습니다. 필요하시면 회의를 통해 자세한 조정안을 함께 논의하고자 합니다. 편하신 시간에 일정 조정 가능하도록 회의 일정도 맞추겠습니다.\n\n감사합니다.\n\n발신자 드림', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1764203452, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=2129, prompt_tokens=60, total_tokens=2189, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=1920, rejected_pr

**Use Case 2: 코드 리뷰**

1.   항목 추가
2.   항목 추가



In [11]:
prompt = '''
다음 python 코드를 리뷰하고 개선점을 제시하세요:

def process_data(data):
    result = []
    for i in range(len(data)):
        if data[i] > 0:
            result.append(data[i])
        return result
'''

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{'role':'user', 'content':prompt}]
)

print(response.choices[0].message.content)

다음 코드는 의도대로 작동하지 않을 가능성이 크고, Pythonic하지 않습니다. 주요 문제점과 개선점을 정리하고, 교체 코드 예시를 제안합니다.

문제점
- 내부 return 위치 버그: return이 for 루프 안에 들여쓰기되어 있어, 첫 번째 반복 후에 함수가 종료됩니다.
- 비효율적이고 덜 Pythonic한 루프: 인덱스로 데이터에 접근하는 방식보다 직접 순회하는 방법이 더 읽기 쉽고 빠릅니다.
- 더 간단한 표현 가능: 리스트 컴프리헨션 등을 활용하면 코드가 간결하고 명확해집니다.
- 선택적 추가: 타입 힌트나 문서화가 없어서 사용처를 파악하기 어렵습니다. 입력 검증이나 예외 처리는 선택사항으로 고려할 수 있습니다.

개선 제안
- 최소 수정: return을 루프 밖으로 이동하고, 인덱스 대신 직접 요소를 순회합니다.
- 더 파이썬답게: 리스트 컴프리헨션으로 한 줄에 구현합니다.
- 타입 힌트와 간단한 문서화 추가를 고려합니다.
- 필요에 따라 제너레이터 버전도 제공 가능.

코드 예시

1) 최소 수정 버전 (버그 수정)
def process_data(data):
    result = []
    for i in range(len(data)):
        if data[i] > 0:
            result.append(data[i])
    return result

수정된 버전(인덱스 대신 직접 순회)
def process_data(data):
    result = []
    for x in data:
        if x > 0:
            result.append(x)
    return result

2) 더 파이썬답고 간결한 버전 (리스트 컴프리헨션)
def process_data(data):
    return [x for x in data if x > 0]

3) 타입 힌트와 간단한 문서화 추가 버전
from typing import Iterable, List, Union

Number = Union[i

In [12]:
# 사용량 확인
print(f"입력 토큰: {response.usage.prompt_tokens}")
print(f"출력 토큰: {response.usage.completion_tokens}")
print(f"총 토큰: {response.usage.total_tokens}")

입력 토큰: 56
출력 토큰: 1945
총 토큰: 2001


In [15]:
from openai import OpenAI
client = OpenAI()

# 매우 긴 system prompt 준비 (약 2000+ 토큰)

best_text = '''
You are an expert AI engineer. Provide concise, correct answers.
This is prefix text for prompt caching demonstration.
Repeat this section many times to increase token length.
'''

# 동일한 문장을 여러 번 반복 >> 길고 반복되는 prefix 생성
long_system_prompt = best_text * 150        # 약 2000~2500 토큰

# 유저메시지
user_message = "이 프롬프트 캐싱 예제에서 system 프롬프트는 몇 글자인가요?"

# 함수: API 호출 + usage 출력

def call_api(tag):
    print(f"\n==={tag} 호출===")
    response = client.chat.completions.create(
        model='gpt-5.1-chat-latest',
        messages=[
            {'role': "system", 'content': long_system_prompt},
            {'role': 'user', 'content': user_message},
        ],
    )

    print(f'입력 토큰 : {response.usage.prompt_tokens}')
    print(f'출력 토큰 : {response.usage.completion_tokens}')
    print(f'총 토큰 : {response.usage.total_tokens}')

    # 캐싱 정보
    details = getattr(response.usage, 'prompt_tokens_details', None)
    print(f'prompt_tokens_details: {details}')

    return response

In [16]:
# 첫 번째 호출(Warm-up : 캐시 생성)
first = call_api('1차 (캐시 생성)')

# 두 번째 호출(Cache Hit : 프롬프트 캐싱 발생)
second = call_api('2차 (캐시 재사용 기대)')


===1차 (캐시 생성) 호출===
입력 토큰 : 4833
출력 토큰 : 1233
총 토큰 : 6066
prompt_tokens_details: PromptTokensDetails(audio_tokens=0, cached_tokens=0)

===2차 (캐시 재사용 기대) 호출===
입력 토큰 : 4833
출력 토큰 : 721
총 토큰 : 5554
prompt_tokens_details: PromptTokensDetails(audio_tokens=0, cached_tokens=3968)


- OpenAI 플랫폼 문서: [프롬프트 캐싱 가이드](https://platform.openai.com/docs/guides/prompt-caching/how-it-works?utm_source=chatgpt.com)
- OpenAI 블로그/제품 페이지 [API 프롬프트 캐싱](https://openai.com/index/api-prompt-caching/?utm_source=chatgpt.com)

#### OpenAI Chat Completion API에서 사용하는 3가지 role 설명
1) system    : 모델의 행동 규칙, 톤, 역할을 설정하는 메시지
2) user      : 사용자 입력(질문, 요구사항 등)을 전달하는 메시지
3) assistant : 모델이 이전에 했던 답변을 다시 포함할 때 사용하는 메시지
---

- 일반적으로 프롬프트를 보낼 때는 "user"를 사용하고,
- 대화 규칙을 설정하려면 "system",
- 이전 모델 답변을 컨텍스트로 넣을 때는 "assistant"를 사용합니다.

## Thinking 모드

### 1. 동적 사고 시간 이해

Thinking 모드의 특징:

- 쉬운 작업: 2배 빠름
- 어려운 작업: 2배 느림 (더 정확)
- reasoning_effort 파라미터로 제어

In [18]:
question = '''
100만 개의 로그 데이터에서 특정 사용자 행동 패턴을 효율적으로 탐지하기 위한 알고리즘을 설계해 주세요.
시간 복잡도, 공간 복잡도, 그리고 왜 이 방법이 최적인지 단계별로 설명해 주세요.
'''

# reasoning_effort 설정
response = client.chat.completions.create(
    model="gpt-5.1",
    messages=[{"role": "user", "content": question}],
    reasoning_effort="low"  # low, medium, high
)

In [19]:
print(response.choices[0].message.content)

아래에서는 “100만 개 로그에서 특정 사용자 행동 패턴(이벤트 시퀀스)을 효율적으로 탐지”하는 전형적인 상황을 가정해서 알고리즘을 설계하겠습니다.

- 로그 한 줄: `(timestamp, user_id, action)`  
- 우리가 찾는 패턴: 하나의 사용자에 대해 시간 순으로 발생한 `k`개의 액션 시퀀스  
  예: `["login", "view_item", "add_to_cart", "checkout"]`  
- 목표: 이 패턴을 수행한 모든 사용자 / 모든 발생 시점 탐지

---

## 1. 전체 접근 개요

1. 로그를 `user_id, timestamp` 기준으로 정렬하여 **사용자별 시간 순 시퀀스**를 만듭니다.
2. 각 사용자 시퀀스에 대해 **선형 시간 패턴 매칭 알고리즘(KMP)**으로 패턴을 찾습니다.
3. 특정 패턴이 아니라 다수의 패턴(예: 여러 시나리오들)을 동시에 찾고 싶다면, 사용자별 시퀀스에 대해 **Aho–Corasick(AC) 자동자**를 사용합니다.

이 접근의 핵심 포인트:
- 로그 수를 `N = 1,000,000`, 패턴 길이를 `k`라고 하면,  
- 전체 시간 복잡도:  
  - 정렬 포함: `O(N log N)` (로그가 섞여 있고 정렬이 필요할 때)  
  - 정렬 생략 가능(이미 user+time 순 정렬): `O(N + k)`  
- 공간 복잡도: `O(N + k)` (또는 AC에서는 패턴 수에 따라 약간 증가)

---

## 2. 단계별 설계

### 2-1. 전처리: 사용자별 시퀀스 만들기

1. 입력: `N`개의 로그 레코드 `(timestamp, user_id, action)`
2. 로그가 이미 `(user_id, timestamp)` 기준으로 정렬되어 있지 않다면:
   - `user_id` → 1차 정렬 키  
   - `timestamp` → 2차 정렬 키로 정렬  
   - 대부분의 DB/로그 시스템에서는 이런 형태로 이미 정렬되어 있는 경우도 많습니다.
3. 정렬 후, 연속된 같은 `